# The partitioning problem

In this exercise we want to solve the so-called *partitioning problem*. This problem takes as input a list, `x`, of length $N$, and your task is to partition `x` into $K$ different contiguous parts such that the *largest* part is as small as possible.

A part is a contiguous block, so the elements of `x` from some index `i` to another `j >= i`. The cost of the block is the sum of the elements 

$$S(i,j) = \sum_{m=i}^{j-1} x[m]$$

In [1]:
# in this function I assume that x is a global variable
def S(i, j):
    return sum(x[m] for m in range(i, j))

In [2]:
x = [2, 5, 3, 7, 5]
for i in range(len(x)):
    for j in range(i, len(x)+1):
        print("S({},{}) == {}".format(i, j, S(i, j)))

S(0,0) == 0
S(0,1) == 2
S(0,2) == 7
S(0,3) == 10
S(0,4) == 17
S(0,5) == 22
S(1,1) == 0
S(1,2) == 5
S(1,3) == 8
S(1,4) == 15
S(1,5) == 20
S(2,2) == 0
S(2,3) == 3
S(2,4) == 10
S(2,5) == 15
S(3,3) == 0
S(3,4) == 7
S(3,5) == 12
S(4,4) == 0
S(4,5) == 5


The cost of a given partitioning is the sum of the elements in the largest part. We are looking for the partitioning with the smallest cost, i.e. the partitioning where the largest cost part is as small as possible.

For example, if `x` is the list 

```python
x = [2, 5, 3, 7, 5]
```

we can split it into two partitions in the following ways:

```python
[] [2, 5, 3, 7, 5]
[2] [5, 3, 7, 5]
[2, 5] [3, 7, 5]
[2, 5, 3] [7, 5]
[2, 5, 3, 7] [5]
[2, 5, 3, 7, 5] []
```

The cost of the first partition is 22, the second 20, the third 15, the fourth 12, the fifth 17, and the sixth 22. The best of these is the fourth with score 12.

If we want to partition `x` into $K$ partitions we can come up with a recursive way to solve this. We can consider where to put the separater between the last partition and the previous partion. At some index `i` into `x`, the last partition must start. The cost of the last index is then `S(i,N)`. The cost of this partitioning must then be

$$\max\left[ P(i,K-1), S(i,N) \right]$$

where $P(i, K-1)$ is the best partitioning of the array `x[0:i]` into $K-1$ partitions. The *best* partitioning of `x` into $K$ partitions is found by picking the optimal index `i`.

$$P(N,K) = \min_{i=0}^{N} \left\{ \max\left[ P(i,K-1), S(i,N) \right] \right\}$$

The basis cases are single partitions, where we have no choice but to put all elements in the same partition

$$P(n,1) = S(0,n)$$

for all $n$, and the empty prefix of `x` where the cost of any number of partitions is zero

$$P(0,k) = 0$$

for all $k$.

## Exercise

Implement a function that computes $P(n,k)$ for any $n=0,\ldots,N$ and $k\geq 0$.

In [3]:
def P(n, k):
    pass

N = len(x)

print(P(N,2), "should be 12")
print(P(N,3), "should be 10")
print(P(N,4), "should be 7")

12 should be 12
10 should be 10
7 should be 7


## Making `S(i,j)` more efficient

In the recursion, we call the `S(i,j)` function many times with the same indices `i` and `j`. Each time, we add up all the values from index `i` and `j`. We can make this more efficient by building a table of these at the beginning and then simply look up the values.

We can build a table using the module `numpy` like this:

In [4]:
import numpy as np

ST = np.zeros((N+1, N+1))
ST

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

Notice the extra set of parentheses when we call `np.zeros`. Those are necessary to get a two-dimensional table. Notice also that we make the size of the table $(N+1) \times (N+1)$. We plan to look up sums that end in index $N$, so we need to be able to index $N$ in the columns, thus we need to allocate one extra column for this.

### Exercise

Write code to fill in the values in the `ST` table.

In [5]:
def fill_ST(N):
    for i in range(N+1):
        for j in range(N+1):
            pass
    
fill_ST(N)
ST

array([[  0.,   2.,   7.,  10.,  17.,  22.],
       [  0.,   0.,   5.,   8.,  15.,  20.],
       [  0.,   0.,   0.,   3.,  10.,  15.],
       [  0.,   0.,   0.,   0.,   7.,  12.],
       [  0.,   0.,   0.,   0.,   0.,   5.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

### Exercise

Write a function `P2` that uses the `ST` table rather than the `S` function.

In [6]:
def P2(n, k):
    pass

print(P2(N,2), "should be 12")
print(P2(N,3), "should be 10")
print(P2(N,4), "should be 7")

12.0 should be 12
10.0 should be 10
7.0 should be 7


We can compare the running time of the two:

In [7]:
%timeit P(N, 3)
%timeit P2(N, 3)

46.5 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
18.8 µs ± 50.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


For larger arrays, and more partitions, the difference should grow

In [8]:
N = 10
x = list(range(N))

ST = np.zeros((N+1, N+1))
fill_ST(N)

K = 5
%timeit P(N, K)
%timeit P2(N, K)

2.22 ms ± 2.63 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
919 µs ± 5.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Building `ST` faster

Filling in all elements in `ST` takes time $O(N^3)$. Can you see why?

We can do slightly better. If we define $CT$ to be the cumulative sum of `x`

$$CT(n) = \sum_{i=0}^{n-1} x[i]$$

we see that $S(i,j) = CT(j)-CT(i)$. So we can build this sum and store it in a table. This makes it faster to compute what should be in each cell in `ST` than it would be to compute the sum for all pairs `i` and `j`.

The `numpy` package has a function for computing the cumulative sum, so we can compute `CT` like this:

In [9]:
np.cumsum(x)

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

This is almost the table we want, but it contains the values up to *and including* index $n$. We want it to not include $n$, so we need to put a zero in front of it. We can get `CT` like this:

In [10]:
CT = np.zeros(N+1)
CT[1:] = np.cumsum(x)
CT

array([  0.,   0.,   1.,   3.,   6.,  10.,  15.,  21.,  28.,  36.,  45.])

I realise that this looks a bit like magic, but it builds a one-dimensional table -- notice that we don't give `zeros` a tuple but a single number -- and it assigns the values from $1$ and up the result of calling `cumsum`.

### Exercise

Write a function, `fill_ST2`, that uses the `CT` table to fill the `ST` table.

In [11]:
def fill_ST2(N):
    for i in range(N):
        for j in range(N+1):
            pass

The `fill_ST` and `fill_ST2` functions should create the same tables

In [12]:
fill_ST(N)
print(ST)

fill_ST2(N)
print(ST)

[[  0.   0.   1.   3.   6.  10.  15.  21.  28.  36.  45.]
 [  0.   0.   1.   3.   6.  10.  15.  21.  28.  36.  45.]
 [  0.   0.   0.   2.   5.   9.  14.  20.  27.  35.  44.]
 [  0.   0.   0.   0.   3.   7.  12.  18.  25.  33.  42.]
 [  0.   0.   0.   0.   0.   4.   9.  15.  22.  30.  39.]
 [  0.   0.   0.   0.   0.   0.   5.  11.  18.  26.  35.]
 [  0.   0.   0.   0.   0.   0.   0.   6.  13.  21.  30.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   7.  15.  24.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   8.  17.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   9.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
[[  0.   0.   1.   3.   6.  10.  15.  21.  28.  36.  45.]
 [  0.   0.   1.   3.   6.  10.  15.  21.  28.  36.  45.]
 [ -1.  -1.   0.   2.   5.   9.  14.  20.  27.  35.  44.]
 [ -3.  -3.  -2.   0.   3.   7.  12.  18.  25.  33.  42.]
 [ -6.  -6.  -5.  -3.   0.   4.   9.  15.  22.  30.  39.]
 [-10. -10.  -9.  -7.  -4.   0.   5.  11.  18.  26.  35.]
 [-15. -15. -

Their running time should be different, though. While `fill_ST(N)` runs in time $O(N^3)$, `fill_ST2(N)` runs in time $O(N^2)$.

In [13]:
%timeit fill_ST(N)
%timeit fill_ST2(N)

98.9 µs ± 440 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
35.4 µs ± 484 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Computing the best partitioning using dynamic programming

Using a table for the `ST` values speeds up the computations because we do not have to recompute sums of contiguous elements repeatedly, but in the recursion we compute the cost of the same partitionings again and again.

We can improve the running time using dynamic programming. Instead of computing $P(N,K)$ recursively, we can fill out a table, `PT`, starting with the basic cases and then build from those, until we can read out the desired value in entry `PT[N,K]`.

In [14]:
def P3(n, k):
    PT = np.zeros((n+1,k+1))
    #fill out the PT table
    return PT[n, k]

The dynamic programming algorithm should give the same results as the other algorithms

In [15]:
for k in range(1, 5):
    print(P(N,k), P2(N,k), P3(N,k))

45 45.0 45.0
24 24.0 24.0
17 17.0 17.0
15 15.0 15.0


Their running time will be different, though

In [16]:
K = 4
%timeit P(N,K)
%timeit P2(N,K)
%timeit P3(N,K)

636 µs ± 1.77 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
244 µs ± 4.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
109 µs ± 400 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
